# Creating Kaggle submissions for CIFAR10 rotation classification (Parts 1/4a)

In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np


def rotate_img(img, rot):
    """
    Implement the same rotation logic as in your code
    """
    if rot == 0: # 0 degrees rotation
        return img
    elif rot == 1:
        return torchvision.transforms.functional.rotate(img, 90)
    elif rot == 2:
        return torchvision.transforms.functional.rotate(img, 180)
    elif rot == 3:
        return torchvision.transforms.functional.rotate(img, 270)
    # TODO: Implement rotate_img() - return the rotated img
    #
    #
    #
    else:
        raise ValueError('rotation should be 0, 90, 180, or 270 degrees')


class CIFAR10RotationTestOnly(torchvision.datasets.CIFAR10):

    def __init__(self, root, download, transform) -> None:
        super().__init__(root=root, train=False, download=download, transform=transform)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int):
        image, cls_label = super().__getitem__(index)
        image_rotated = rotate_img(image, index % 4)

        return image_rotated

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 128

testset = CIFAR10RotationTestOnly(root='./data', download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
@torch.inference_mode()
def run_rotation_test(net, testloader):
    all_output = []
    for images in testloader:
        images = images.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        all_output.append(predicted.cpu().numpy())
    return np.concatenate(all_output)

## Instantiate your model here

In [17]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

net = models.densenet121(num_classes=4)
net = net.to(device)
net.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

## Load the model weights

In [18]:
net.load_state_dict(torch.load('DenseNet_rotation'))

<All keys matched successfully>

## Submit the generated csv file to Kaggle

In [19]:
import pandas as pd
predictions = run_rotation_test(net, testloader)
ids = np.arange(len(predictions))
df = pd.DataFrame({'id': ids, 'prediction': predictions})
df.to_csv('./rotation_test_predictions_part4.csv', index=False)